In [9]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
from datetime import date

In [2]:
def get_game(url):
    random_number = random.randint(0, 5)
    time.sleep(random_number)
    bets = []
    service = Service(executable_path="chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    time.sleep(8) 

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    game_title = soup.find('span', class_='o_match-title').text.strip()
   
    accordion_button = driver.find_element(By.XPATH, "//button[@class='o_accordion-item__trigger']//span[contains(text(), 'Genaues Ergebnis')]")
    accordion_button.click()

    time.sleep(2)  

    accordion_item = accordion_button.find_element(By.XPATH, "./ancestor::div[contains(@class, 'o_accordion-item')]")

    if 'o_accordion-item--expanded' in accordion_item.get_attribute('class'):
        panel = accordion_item.find_element(By.CLASS_NAME, 'o_accordion-item__panel')
        panel_html_content = panel.get_attribute('innerHTML')
        panel_soup = BeautifulSoup(panel_html_content, 'html.parser')
        bet_items_section = panel_soup.find_all('div', class_='o_bet-item')

        for section in bet_items_section:
            header = section.find('div', class_='o_bet-item__header').text.strip()
            if "Genaues Resultat (reg.Spielzeit)" in header:
                bet_items = section.find('div', class_='o_bet-item__content ng-star-inserted').find_all('button', class_='m_bet-button')
                for item in bet_items:
                    name = item.find('span', class_='m_bet-button__name').text.strip()
                    odds = item.find('span', class_='m_bet-button__odds-text').text.strip()
                    bets.append((name, odds))
                break
    else:
        print("Accordion item is not expanded.")


    driver.quit()
    return bets,game_title
    

In [3]:
all_games = []
with open('list_urls.txt', 'r') as file:
    all_games = [line.strip() for line in file.readlines()]

In [4]:
def call_five_times(url): 
    bets = []
    game_title = ""
    for i in range(5):
        try: 
            bets, game_title = get_game(url)
            if bets:
                print("succeeded ",game_title)
                return bets, game_title
        except Exception as e:
            print(f"Attempt {i+1} failed for game {url}")

In [5]:
all_bets_data = []
failed = []
with ThreadPoolExecutor(max_workers=3) as executor:
    future_to_game = {executor.submit(call_five_times, url): url for url in all_games}

    for i, future in enumerate(as_completed(future_to_game)):
        game_url = future_to_game[future]
        try:
            bets, game_title = future.result()
            if bets:
                for name, odds in bets:
                    all_bets_data.append([game_title, name, float(odds)])
            else:
                failed.append(game_url)
        except Exception as e:
            print(f"Error occurred for game {game_url}: {e}")
            failed.append(game_url)



Attempt 1 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2631/event/497241
Attempt 1 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2631/event/497257
succeeded  Spanien – Kroatien
Attempt 2 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2631/event/497241
Attempt 2 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2631/event/497257
Attempt 1 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2632/event/497258
Attempt 3 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2631/event/497241
succeeded  Deutschland – Schottland
succeeded  Italien – Albanien
Attempt 4 failed for g

In [8]:
# try again failed ones 
for f in failed:
    bets, game_title = call_five_times(f)
    if bets:
        for name, odds in bets:
            all_bets_data.append([game_title, name, float(odds)])
    
    

succeeded  Slowenien – Dänemark


In [17]:
df = pd.DataFrame(all_bets_data, columns=["Game Title", "Result", "Odds"])
# Sort the DataFrame by game title and odds
df_sorted = df.sort_values(by=["Game Title", "Odds"])

# Keep only the 3 lowest odds for each game title
df_filtered = df_sorted.groupby("Game Title").head(3)

df_one = df_sorted.groupby("Game Title").head(1)
df_filtered = df_filtered.sort_index()

# Display the filtered DataFrame
display(df_filtered)

today = date.today().strftime("%d_%m_%Y")
df_filtered.to_csv("odds/"+f"odds_{today}.csv")


,Game Title,Result,Odds
0,Spanien – Kroatien,1:0,5.70
1,Spanien – Kroatien,2:0,6.99
13,Spanien – Kroatien,1:1,5.50
29,Deutschland – Schottland,1:0,5.84
30,Deutschland – Schottland,2:0,5.11
...,...,...,...
999,Tschechische Republik – Türkei,1:1,5.13
1002,Tschechische Republik – Türkei,0:1,7.38
1028,Slowenien – Dänemark,1:1,5.67
1031,Slowenien – Dänemark,0:1,5.17


In [19]:
df_one.sort_index()

,Game Title,Result,Odds
13,Spanien – Kroatien,1:1,5.50
30,Deutschland – Schottland,2:0,5.11
58,Italien – Albanien,1:0,4.69
100,Ungarn – Schweiz,1:1,5.27
132,Serbien – England,0:1,5.76
161,Polen – Niederlande,0:1,5.63
187,Rumänien – Ukraine,1:1,5.47
204,Belgien – Slowakei,2:0,6.05
249,Österreich – Frankreich,0:2,6.42
261,Türkei – Georgien,1:0,5.07
